# Proyecto mineria de datos

coordinates plot libraries

In [1]:
import json
from pprint import pprint

import numpy as np
import matplotlib.pyplot as plt

import matplotlib as mpl
%matplotlib inline
mpl.style.use('ggplot')

from scipy.misc import imread
from mpl_toolkits.basemap import Basemap



Text mining libraries

In [2]:
import codecs
import glob
import json
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import re
import string
import time

#nltk.download()
%matplotlib inline
from ggplot import *
from math import log
from matplotlib import pyplot as plt
from nltk import sent_tokenize, word_tokenize
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from scipy import ndimage
from sklearn import manifold, datasets
from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from time import time

## Load

In [3]:
with open('test2.json') as data_file:    
    #data = json.load(data_file, encoding="latin-1")
    data = json.load(data_file, encoding="utf-8")

### Data structure

In [4]:
print "páginas de registros:", len(data)
print "claves por página:", data[0].keys()
print "claves de \"photos\":" ,data[0]["photos"].keys()
print "fotos por página", len(data[0]["photos"]["photo"])
print "claves por \"photo\"",pd.DataFrame( data[0]["photos"]["photo"][0].keys() )

páginas de registros: 500
claves por página: [u'photos', u'stat']
claves de "photos": [u'photo', u'total', u'pages', u'page', u'perpage']
fotos por página 250
claves por "photo"                        0
0              pathalias
1          geo_is_public
2               place_id
3           media_status
4                  owner
5                     id
6   datetakengranularity
7                  woeid
8          geo_is_friend
9         geo_is_contact
10             datetaken
11              isfriend
12                secret
13             ownername
14              latitude
15              accuracy
16              isfamily
17                 media
18           description
19                  tags
20                  farm
21         geo_is_family
22      datetakenunknown
23              ispublic
24                 url_s
25             longitude
26                server
27               context
28              height_s
29                 title
30               width_s


# Preprocessing

## Plot coordinates of the pictures

In [5]:
Lat=[]
Long=[]
Tags=[]
Ids=[]
Urls=[]
for datas in data:
    Lat_datas=[float(pic["latitude"]) for pic in datas["photos"]["photo"]]
    Long_datas=[float(pic["longitude"]) for pic in datas["photos"]["photo"]]
    Tags_datas=[pic["tags"] for pic in datas["photos"]["photo"]]
    Ids_datas=[pic["id"] for pic in datas["photos"]["photo"]]
    Urls_datas=[pic["url_s"] for pic in datas["photos"]["photo"]]
    
    Lat.extend(Lat_datas)
    Long.extend(Long_datas)
    Tags.extend(Tags_datas)
    Ids.extend(Ids_datas)
    Urls.extend(Urls_datas)

### Plot pictures coordintaes in a map

https://stevendkay.wordpress.com/tag/basemap/

https://www.getdatajoy.com/examples/python-plots/plot-data-points-on-a-map


#### Basemap documentation
https://basemaptutorial.readthedocs.io/en/latest/

http://matplotlib.org/basemap/users/index.html

http://matplotlib.org/basemap/users/examples.html


#### load picture method
http://stackoverflow.com/questions/15160123/adding-a-background-image-to-a-plot-with-known-corner-coordinates

https://www.openstreetmap.org/

### Bogotá coordinates

East long=-73.98

West long=-74.23

North lat=4.84

South lat=4.46

In [ ]:
North=4.84
South=4.46
East=-73.98
West=-74.23

def map_dim(wide):
    high_m=abs(North-South)
    wide_m=abs(East-West)
    ratio=high_m/wide_m
    return([float(wide),float(wide)*ratio])

In [ ]:
map_dim(15)

In [ ]:
img = imread("map2.jpg")
plt.figure(figsize=(map_dim(15)[0],map_dim(15)[1]))
plt.imshow(img,zorder=1, extent=[-74.23, -73.98,4.46,  4.84])
plt.scatter(Long, Lat, alpha=0.1,zorder=2)
plt.ylim(4.46,4.84)
plt.xlim(-74.23, -73.98)
plt.savefig('mapa_puntos.jpg')
plt.show()


## Tags processing

In [ ]:
print len(list(set(Ids)))
print len(list(set(Tags)))
print len(list(set(Urls)))

In [6]:
#Tags
#Tags[7].encode('utf-8').decode('utf-8')
print Tags[6]

bogota bicycle cicla bicicleta street ciclorruta city bike bici fotografía fotografiacallejera photography cycle fotografia streetphotography colombia bogotá fotografíacallejera ciudad co


## Tf-idf matrix 

In [14]:
tokens_list = []
counter = 0

regex_str = [
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

regex_rem = [
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
tokens_remove = re.compile(r'('+'|'.join(regex_rem)+')', re.VERBOSE | re.IGNORECASE)

punctuation = list(string.punctuation)
stopEn = stopwords.words('english') + punctuation + ['rt', 'via']
stopEs = stopwords.words('spanish') + punctuation + ['rt', 'via']

def tokenize(s):
    return tokens_re.findall(s)
          
def preprocess(s, lowercase=True):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token.lower() for token in tokens]
    return tokens

#with open(path+'iot_data_130516.txt', 'r') as f:
for line in Tags:
    counter+=1
    if counter == 2000:
        break;
    tag = line
    tagString = ""
    for term in preprocess(tag):
        if not tokens_remove.search(term) and term not in stopEn and term not in stopEs:
            tagString = tagString + term + " "
    tokens_list.append(tagString)
    
print counter

tfidf = TfidfVectorizer(max_df=0.5, min_df=2)
tfs = tfidf.fit_transform(tokens_list)

print "Dimensiones tf-idf",tfs.shape
print tfidf
print tfidf.idf_
print tfs    

  

2000
Dimensiones tf-idf (1999, 379)
TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.5, max_features=None, min_df=2,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)
[ 4.9633163   6.80914299  6.80914299  7.50229017  6.80914299  6.29831737
  7.50229017  3.92807363  7.2146081   7.50229017  6.52146092  6.99146455
  7.50229017  6.80914299  6.80914299  7.50229017  7.50229017  6.65499231
  7.50229017  3.35915544  5.38202663  6.20300719  7.50229017  6.29831737
  6.99146455  7.2146081   7.2146081   6.80914299  6.80914299  7.50229017
  6.40367788  6.99146455  7.50229017  6.80914299  7.50229017  7.50229017
  7.50229017  7.50229017  7.50229017  7.50229017  7.2146081   7.2146081

### With english stopwords

In [15]:
tfidf = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english')
tfs = tfidf.fit_transform(tokens_list)

print "Dimensiones tf-idf",tfs.shape
print tfidf
print tfidf.idf_
print tfs  

Dimensiones tf-idf (1999, 380)
TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.5, max_features=None, min_df=2,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)
[ 4.9633163   6.80914299  6.80914299  7.50229017  6.80914299  6.29831737
  7.50229017  3.92807363  7.2146081   7.50229017  6.52146092  6.99146455
  7.50229017  6.80914299  6.80914299  7.50229017  7.50229017  6.65499231
  7.50229017  3.35915544  5.38202663  6.20300719  7.50229017  6.29831737
  6.99146455  7.2146081   7.2146081   6.80914299  6.80914299  7.50229017
  6.40367788  6.99146455  7.50229017  6.80914299  7.50229017  7.50229017
  7.50229017  7.50229017  7.50229017  7.50229017  7.2146081   7.2146081
  6.80914

In [16]:
#Tokens
tokens_list

['',
 '',
 '',
 '',
 '',
 '',
 u'bogota bicycle cicla bicicleta street ciclorruta city bike bici fotograf \xed fotografiacallejera photography cycle fotografia streetphotography colombia bogot \xe1 fotograf \xed acallejera ciudad co ',
 u'bogot \xe1 colombia co lights townatnight landscape canon aire libre cielo ngc ',
 '',
 '',
 '',
 u'instagramapp square squareformat iphoneography uploaded instagram ',
 u'instagramapp square squareformat iphoneography uploaded instagram ',
 u'instagramapp square squareformat iphoneography uploaded instagram ',
 u'instagramapp square squareformat iphoneography uploaded instagram ',
 u'fujixt flickr cabaret wb contrast cafe city lights carpa paraiso fachada ciudad bogota fujifilm lightbulb bombillas ',
 u'instagramapp square squareformat iphoneography uploaded instagram clarendon ',
 u'instagramapp square squareformat iphoneography uploaded instagram ',
 u'instagramapp square squareformat iphoneography uploaded instagram ',
 u'carrera septima bogota co